## Import libraries

In [67]:
import numpy as np
import pandas as pd


In [68]:
df = pd.read_excel("~/Personal_project/Customer-Lifetime-Value-Prediction/data/Online_Retail.xlsx")

In [69]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Understanding the data

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [71]:
##Looking for null values
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [72]:
## looking for duplicates
print("Before dropping null values: ", df.shape)
df.duplicated().sum()
# Dropping the null values
df.dropna(inplace=True)


print("After dropping null values: ", df.shape)

Before dropping null values:  (541909, 8)
After dropping null values:  (406829, 8)


In [73]:
df.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [74]:
## Dropping null values
df.dropna(inplace=True)

print("After dropping null values: ", df.shape)

After dropping null values:  (406829, 8)


In [75]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,406829.000000,406829,406829.000000,406829.000000
mean,12.061303,2011-07-10 16:30:57.879207424,3.460471,15287.690570
min,-80995.000000,2010-12-01 08:26:00,0.000000,12346.000000
25%,2.000000,2011-04-06 15:02:00,1.250000,13953.000000
50%,5.000000,2011-07-31 11:48:00,1.950000,15152.000000
75%,12.000000,2011-10-20 13:06:00,3.750000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,248.693370,NaN,69.315162,1713.600303


In [76]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [77]:
columns = ['StockCode', 'Description', 'Quantity','UnitPrice', 'CustomerID', 'Country']

for col in columns:
    print(f'Unique values in {col} : {df[col].unique()} \n\n Number of unique values in {col} : {len(df[col].unique())}\n\n')

Unique values in StockCode : ['85123A' 71053 '84406B' ... '90214Z' 90089 23843] 

 Number of unique values in StockCode : 3684


Unique values in Description : ['WHITE HANGING HEART T-LIGHT HOLDER' 'WHITE METAL LANTERN'
 'CREAM CUPID HEARTS COAT HANGER' ... 'PINK CRYSTAL SKULL PHONE CHARM'
 'CREAM HANGING HEART T-LIGHT HOLDER' 'PAPER CRAFT , LITTLE BIRDIE'] 

 Number of unique values in Description : 3896


Unique values in Quantity : [     6      8      2     32      3      4     24     12     48     18
     20     36     80     64     10    120     96     23      5      1
     -1     50     40    100    192    432    144    288    -12    -24
     16      9    128     25     30     28      7     72    200    600
    480     -6     14     -2     -4     -5     -7     -3     11     70
    252     60    216    384     27    108     52  -9360     75    270
     42    240     90    320     17   1824    204     69    -36   -192
   -144    160   2880   1400     19     39    -48    -50     56 

In [78]:
customer_df =df.copy()

### Changing datatype and remove duplicates

In [79]:
customer_df['CustomerID'] = customer_df['CustomerID'].astype(str)
customer_df['InvoiceDate'] = pd.to_datetime(customer_df['InvoiceDate'])
customer_df['Quantity'] = customer_df['Quantity'].abs()

In [80]:
object_cols = ['InvoiceNo','StockCode']
for col in object_cols:
    customer_df[col] = pd.to_numeric(customer_df[col], errors='coerce')


In [81]:
## looking for duplicates
print("Before dropping null values: ", customer_df.shape)
customer_df.duplicated().sum()
# Dropping the null values
customer_df.dropna(inplace=True)


print("After dropping null values: ", customer_df.shape)

Before dropping null values:  (406829, 8)
After dropping null values:  (363119, 8)


## Create a Total price Column

In [82]:
customer_df['TotalPrice'] = customer_df['Quantity'] * customer_df['UnitPrice']

In [83]:
customer_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
1,536365.0,71053.0,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
5,536365.0,22752.0,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom,15.30
6,536365.0,21730.0,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,25.50
7,536366.0,22633.0,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10
8,536366.0,22632.0,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10


In [84]:
customer_df.shape

(363119, 9)

## Feature Engineering

**Recency**: Days since last purchase.

It uses ''' groupby + .diff() ''' which computes:

InvoiceDate(t) - InvoiceDate(t-1)

In [85]:
customer_df['InvoiceDate'] = pd.to_datetime(customer_df['InvoiceDate'])

customer_df = customer_df.sort_values(by= ['CustomerID', 'InvoiceDate'])

customer_df['Recency'] = customer_df.groupby('CustomerID')['InvoiceDate'].diff().dt.days

In [86]:
customer_df['Recency'] = customer_df['Recency'].fillna(0)

**Frequency**: Total number of purchases.

In [87]:
customer_df ['Frequency'] = customer_df.groupby('CustomerID')['InvoiceDate'].transform('count')

**Monetary**: Total revenue generated.[Total spending per customer]

In [88]:
# Group by CustomerID and sum TotalPrice
monetary_df = customer_df.groupby('CustomerID')['TotalPrice'].sum().reset_index(name='Monetary')

# Merge back to original DataFrame
customer_df = customer_df.merge(monetary_df, on='CustomerID', how='left')


**Customer Lifetime**: Duration from first to last purchase.

In [89]:
customer_df['InvoiceDate'] = pd.to_datetime(customer_df['InvoiceDate'])

# Group by customer to get first and last purchase dates
customer_lifetime = customer_df.groupby('CustomerID').agg(
    first_purchase=('InvoiceDate', 'min'),
    last_purchase=('InvoiceDate', 'max')
)

# Calculate lifetime in days
customer_lifetime['customer_lifetime_days'] = (
    customer_lifetime['last_purchase'] - customer_lifetime['first_purchase']
).dt.days

# Merge this info back to the original DataFrame if needed
customer_df = customer_df.merge(customer_lifetime[['customer_lifetime_days']], on='CustomerID', how='left')


In [90]:
print(f"Number of unique values in customer_lifetime_days column: {customer_df['customer_lifetime_days'].unique().sum()}")

Number of unique values in customer_lifetime_days column: 69751


#### 🔸 1. **CLV (Customer Lifetime Value):**
Use Frequency, Monetary, and Customer Lifetime to segment high- vs. low-value customers.

**Basic formula:**

CLV = Average Order Value × Purchase Frequency × Customer Lifetime

CLV=Average Order Value×Purchase Frequency×Customer Lifetime
#### 🔸 2. **Churn Probability:**
Longer lifetimes suggest higher engagement. Sudden drops in frequency could imply churn risk.

#### 🔸 3. **Cohort Analysis:**
Analyze how different customer acquisition months result in different average lifetimes.

In [91]:
customer_df.isnull().sum()

InvoiceNo                 0
StockCode                 0
Description               0
Quantity                  0
InvoiceDate               0
UnitPrice                 0
CustomerID                0
Country                   0
TotalPrice                0
Recency                   0
Frequency                 0
Monetary                  0
customer_lifetime_days    0
dtype: int64

In [92]:
customer_df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Recency,Frequency,Monetary,customer_lifetime_days
0,541431.0,23166.0,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.6,0.0,1,77183.60,0
1,537626.0,85116.0,BLACK CANDELABRA T-LIGHT HOLDER,12,2010-12-07 14:57:00,2.10,12347.0,Iceland,25.2,0.0,162,3653.45,365
2,537626.0,22375.0,AIRLINE BAG VINTAGE JET SET BROWN,4,2010-12-07 14:57:00,4.25,12347.0,Iceland,17.0,0.0,162,3653.45,365
3,537626.0,71477.0,COLOUR GLASS. STAR T-LIGHT HOLDER,12,2010-12-07 14:57:00,3.25,12347.0,Iceland,39.0,0.0,162,3653.45,365
4,537626.0,22492.0,MINI PAINT SET VINTAGE,36,2010-12-07 14:57:00,0.65,12347.0,Iceland,23.4,0.0,162,3653.45,365


## Label Encoding 

In [93]:
from sklearn.preprocessing import LabelEncoder

# For Country column
le = LabelEncoder()
customer_df['Country_Label'] = le.fit_transform(customer_df['Country'])


#### For **Description** column 
TF-IDF converts text to numeric vectors that represent how important a word is in a document relative to a collection (corpus).

TF (term frequency): How often a term appears in the product description.

IDF (inverse document frequency): How rare the term is across all descriptions.

Result: Words that appear often in one product but rarely across others get a higher weight.

#### What happening below code:
1.Preprocess the Description column (handle missing text).

2.Generate a TF-IDF matrix with max_features=100.

3.Apply Truncated SVD (a PCA-like dimensionality reduction for sparse data).

4.Merge the resulting topic features back into your original customer_df.

In [94]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Step 1: Fill missing descriptions with empty string
customer_df['Description'] = customer_df['Description'].fillna('')

# Step 2: Create TF-IDF features
tfidf = TfidfVectorizer(
    max_features=100,        # top 100 terms
    stop_words='english',    # remove common English stopwords
    ngram_range=(1, 2),      # use unigrams and bigrams
    min_df=5,                # ignore rare words
    max_df=0.9               # ignore very common words
)
desc_tfidf = tfidf.fit_transform(customer_df['Description'])

# Step 3: Reduce dimensions with Truncated SVD
n_topics = 20  # can be adjusted between 10–20 as needed
svd = TruncatedSVD(n_components=n_topics, random_state=42)
desc_topics = svd.fit_transform(desc_tfidf)

# Step 4: Create a DataFrame from SVD results
desc_topic_df = pd.DataFrame(
    desc_topics,
    columns=[f'desc_topic_{i+1}' for i in range(n_topics)]
)

# Step 5: Concatenate topics with the original customer_df
customer_df = customer_df.reset_index(drop=True)
customer_df = pd.concat([customer_df, desc_topic_df], axis=1)

# Optional: drop the original Description column if no longer needed
# customer_df.drop('Description', axis=1, inplace=True)

# Preview final data
customer_df.head()



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Recency,...,desc_topic_11,desc_topic_12,desc_topic_13,desc_topic_14,desc_topic_15,desc_topic_16,desc_topic_17,desc_topic_18,desc_topic_19,desc_topic_20
0,541431.0,23166.0,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.6,0.0,...,0.045034,-0.026093,-0.048312,0.064145,-0.009803,-0.003681,-0.012714,-0.015699,0.055104,-0.003688
1,537626.0,85116.0,BLACK CANDELABRA T-LIGHT HOLDER,12,2010-12-07 14:57:00,2.10,12347.0,Iceland,25.2,0.0,...,0.406481,-0.240600,-0.322432,0.468263,0.131622,-0.058090,0.034467,-0.090115,0.036342,0.050075
2,537626.0,22375.0,AIRLINE BAG VINTAGE JET SET BROWN,4,2010-12-07 14:57:00,4.25,12347.0,Iceland,17.0,0.0,...,-0.111430,-0.007065,-0.000049,0.023634,0.138655,-0.031094,0.066135,-0.042782,0.062202,-0.060593
3,537626.0,71477.0,COLOUR GLASS. STAR T-LIGHT HOLDER,12,2010-12-07 14:57:00,3.25,12347.0,Iceland,39.0,0.0,...,0.380712,-0.209338,-0.306541,0.453121,0.123219,-0.037794,0.041952,-0.082633,0.030501,0.078181
4,537626.0,22492.0,MINI PAINT SET VINTAGE,36,2010-12-07 14:57:00,0.65,12347.0,Iceland,23.4,0.0,...,-0.094291,-0.032696,0.018261,0.032143,0.108406,0.001612,0.074616,-0.024091,-0.016158,-0.066717


**Output Explanation**

After running the code, your customer_df will now include:

All original columns (e.g., InvoiceNo, CustomerID, etc.)

20 new columns: desc_topic_1 to desc_topic_20

These are semantic topics extracted from product descriptions.

Each column is a latent dimension that captures hidden patterns (e.g., gift sets, kitchen items, seasonal goods, etc.)

***svd.components_*** is a powerful way to understand what each latent topic actually means in terms of product descriptions.

In [95]:
#Get TF-IDF Feature Names
feature_names = tfidf.get_feature_names_out()
#Get Top Words per Topic
n_top_words = 10

for topic_idx, component in enumerate(svd.components_):
    top_indices = component.argsort()[::-1][:n_top_words]
    top_words = [feature_names[i] for i in top_indices]
    print(f"Topic #{topic_idx + 1}: {', '.join(top_words)}")



Topic #1: set, retrospot, red, bag, red retrospot, design, vintage, heart, paper, tins
Topic #2: bag, red, retrospot, lunch, red retrospot, jumbo, lunch bag, vintage, design, jumbo bag
Topic #3: red, metal, retrospot, sign, metal sign, red retrospot, heart, small, decoration, hanging
Topic #4: metal, sign, metal sign, bag, design, blue, jumbo, vintage, jumbo bag, lunch
Topic #5: heart, christmas, vintage, decoration, wicker, hanging, small, white, large, holder
Topic #6: vintage, christmas, vintage christmas, tin, red, card, kit, paper, jumbo, paisley
Topic #7: box, vintage, design, christmas, lunch, blue, pantry, ceramic, spaceboy, cake
Topic #8: design, christmas, cake, pack, pantry, cases, cake cases, spaceboy, lunch bag, hand
Topic #9: christmas, wooden, 50, box, bag, star, paper, lunch, retrospot, decoration
Topic #10: pink, pack, cake, cases, cake cases, regency, christmas, 60, 12, polkadot
Topic #11: card, pink, light, holder, birthday, light holder, glass, regency, hanging, pol

#### Create a Topic Label Dictionary

In [96]:
topic_keywords = {}
for topic_idx, component in enumerate(svd.components_):
    top_indices = component.argsort()[::-1][:n_top_words]
    top_words = [feature_names[i] for i in top_indices]
    topic_keywords[f'desc_topic_{topic_idx + 1}'] = top_words

# Optional: convert to DataFrame
topic_label_df = pd.DataFrame.from_dict(topic_keywords, orient='index', columns=[f'word_{i+1}' for i in range(n_top_words)])
print(topic_label_df)


                  word_1     word_2             word_3      word_4  \
desc_topic_1         set  retrospot                red         bag   
desc_topic_2         bag        red          retrospot       lunch   
desc_topic_3         red      metal          retrospot        sign   
desc_topic_4       metal       sign         metal sign         bag   
desc_topic_5       heart  christmas            vintage  decoration   
desc_topic_6     vintage  christmas  vintage christmas         tin   
desc_topic_7         box    vintage             design   christmas   
desc_topic_8      design  christmas               cake        pack   
desc_topic_9   christmas     wooden                 50         box   
desc_topic_10       pink       pack               cake       cases   
desc_topic_11       card       pink              light      holder   
desc_topic_12       card     bottle              water         hot   
desc_topic_13       pink     bottle                hot       water   
desc_topic_14      l

### Date Feature Engineering

In [97]:
customer_df['InvoiceDate'] = pd.to_datetime(customer_df['InvoiceDate'])

customer_df['InvoiceYear'] = customer_df['InvoiceDate'].dt.year
customer_df['InvoiceMonth'] = customer_df['InvoiceDate'].dt.month
customer_df['InvoiceDay'] = customer_df['InvoiceDate'].dt.day
customer_df['InvoiceHour'] = customer_df['InvoiceDate'].dt.hour
customer_df['Weekday'] = customer_df['InvoiceDate'].dt.weekday


### Drop Unwanted Columns


In [98]:
customer_df.drop(columns=['Description','Country'],inplace=True)

print('Shape of Final dataset:', customer_df.shape)


Shape of Final dataset: (363119, 37)


In [99]:
customer_df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,TotalPrice,Recency,Frequency,Monetary,...,desc_topic_16,desc_topic_17,desc_topic_18,desc_topic_19,desc_topic_20,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceHour,Weekday
0,541431.0,23166.0,74215,2011-01-18 10:01:00,1.04,12346.0,77183.6,0.0,1,77183.60,...,-0.003681,-0.012714,-0.015699,0.055104,-0.003688,2011,1,18,10,1
1,537626.0,85116.0,12,2010-12-07 14:57:00,2.10,12347.0,25.2,0.0,162,3653.45,...,-0.058090,0.034467,-0.090115,0.036342,0.050075,2010,12,7,14,1
2,537626.0,22375.0,4,2010-12-07 14:57:00,4.25,12347.0,17.0,0.0,162,3653.45,...,-0.031094,0.066135,-0.042782,0.062202,-0.060593,2010,12,7,14,1
3,537626.0,71477.0,12,2010-12-07 14:57:00,3.25,12347.0,39.0,0.0,162,3653.45,...,-0.037794,0.041952,-0.082633,0.030501,0.078181,2010,12,7,14,1
4,537626.0,22492.0,36,2010-12-07 14:57:00,0.65,12347.0,23.4,0.0,162,3653.45,...,0.001612,0.074616,-0.024091,-0.016158,-0.066717,2010,12,7,14,1


## Save Final Dataset 

In [100]:
customer_df.to_csv("~/Personal_project/Customer-Lifetime-Value-Prediction/data/final_dataset.csv", index=False)